# Outlier Method of Filtering

**Mostafa Kamal Mostafa Kamel (mos.kamal@nu.edu.eg)**


## Eval


In [ ]:
from seasalt.seasalt import apply_salt_pepper
from PIL import Image
from glob import glob
from pathlib import Path
import pandas as pd
from typing import Callable, Optional
from skimage.metrics import structural_similarity, peak_signal_noise_ratio


import matplotlib.pyplot as plt
import numpy as np
from rich.progress import Progress

from scipy.signal import medfilt
import warnings
import seaborn as sns
from seasalt.cython_seasalt import fixed_window_outlier_filter

warnings.filterwarnings("ignore")
path_to_images = Path().resolve().joinpath("BSD68")
image_paths = glob(f"{str(path_to_images)}/*")

def plot_transformation_hist_eq(
    im, sp_ratio, size=3, max_size=15, metric="PSNR", save_path: Optional[str] = None
) -> float:
    metric_calculator = peak_signal_noise_ratio if metric=="PSNR" else structural_similarity
    im_gs = im.convert("L")
    arr = np.array(im_gs)  # convert the PIL image object to array
    seasoned_image = apply_salt_pepper(arr, ratio=sp_ratio)
    corrected_novel = fixed_window_outlier_filter(np.copy(seasoned_image), size)
    noisy_metric = metric_calculator(arr, seasoned_image)
    novel_metric = metric_calculator(arr, corrected_novel)
    fig = plt.figure(
        figsize=(
            12*3,
            5*3,
        ),
        dpi=80,
    )
    gs = fig.add_gridspec(1, 3, hspace=0.12, wspace=0.08)
    axes = gs.subplots(sharex="col", sharey="row")
    list(map(lambda ax: ax.set_axis_off(), axes.flatten()))
    list(  # type: ignore
        map(
            lambda input: input[0].set_title(input[1]),
            zip(
                axes.flatten(),
                [
                    "Original",
                    f"Noisy Image {round(noisy_metric, 2)} dB",
                    "Novel Approach\n" f"{round(novel_metric, 2)} dB",
                ],
            ),
        )
    )
    axes[0].imshow(im_gs, cmap="gray", vmin=0, vmax=255)
    axes[1].imshow(seasoned_image, cmap="gray", vmin=0, vmax=255)
    axes[2].imshow(corrected_novel, cmap="gray", vmin=0, vmax=255)
    if save_path:
        plt.axis("off")
        Path(f"./eval/BSD58-test/{metric}/{save_path}").parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(f"./eval/BSD58-test/{metric}/{save_path}")
        plt.close()
    return novel_metric

In [ ]:
def plot_single_image(seasoned_arr):
    fig = plt.figure(
        figsize=(
            10,
            20,
        ),
        dpi=80,
    )
    gs = fig.add_gridspec(1, 1, hspace=0.12, wspace=0.08)
    ax = gs.subplots(sharex="col", sharey="row")
    ax.imshow(seasoned_arr, cmap="gray", vmin=0, vmax=255)
    ax.set_axis_off()

In [ ]:
data_dict_eq = dict(
    im_index=[],
    sp=[],
    novel_snr=[],
)
root_path = Path().resolve().joinpath("aof_eval").joinpath("eq")

with Progress() as progress:
    task = progress.add_task("Running...", total=len(image_paths) * 10)

    for p in image_paths:
        img_name = p.split("/")[-1].split(".")[0]
        for sp, size in zip(
            [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95],
            [3,   3,   3,   5,   5,   5,   7,   11,   17,   19],
        ):
            im = Image.open(p)
            novel_snr = plot_transformation_hist_eq(
                im,
                sp_ratio=sp,
                size=size,
                metric="PSNR",
                save_path=f"{p.split('/')[-1].split('.')[0]}/{int(sp*100)}.png",
            )
            data_dict_eq["im_index"].append(img_name)
            data_dict_eq["sp"].append(sp)
            data_dict_eq["novel_snr"].append(novel_snr)
            progress.update(task, advance=1)

In [ ]:
sns.set_theme()

In [ ]:
df_eq = pd.DataFrame(data_dict_eq)
df_eq.to_pickle("df_eq_BSD68_PSNR.pkl")

In [ ]:
df_aof_agg = df_eq.groupby("sp").agg(
    {
        "novel_snr": [np.mean],
    }
)

In [ ]:
df_aof_agg = df_aof_agg.reset_index()
df_aof_agg.columns = [
    "sp",
    "mean_novel_snr",
]

In [ ]:
df_aof_agg

In [ ]:
df_melted = df_aof_agg.melt(id_vars="sp", var_name="variable", value_name="value")
df_melted

In [ ]:
df_melted.to_clipboard()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=df_melted[
        df_melted.variable.isin(
            [
                "mean_novel_snr",
            ]
        )
    ],
    x="sp",
    y="value",
    hue="variable",
)

plt.xlabel("Salt and Pepper Ratio")
plt.ylabel("SNR")
plt.show()